In [ ]:
import tkinter as tk
import cv2
import pytesseract
import re
import mysql.connector
import numpy as np
from PIL import Image
from tkinter import filedialog, messagebox, ttk
from pdf2image import convert_from_path
import os

# Ajouter l'import
from datetime import datetime

# Chemin vers l'exécutable de Tesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

TESSERACT_LANG = 'deu+eng'  # Allemand

# Fonctions de prétraitement (inchangées)
def choose_file():
    tk.Tk().withdraw()
    return filedialog.askopenfilename(
        title="Rechnung auswählen",
        filetypes=[("Images und PDFs", "*.jpg *.jpeg *.png *.pdf")]
    )

#Prétraitement de l'image pour l'amélioration de l'OCR
def preprocess_image(file_path):
    filename = os.path.basename(file_path)
    name, ext = os.path.splitext(filename)
    output_folder = 'preprocessed'
    os.makedirs(output_folder, exist_ok=True)

    if ext.lower() == '.pdf':
        # Convertir la première page du PDF en image
        pages = convert_from_path(file_path)
        page = pages[0]
        image_path = os.path.join(output_folder, f"{name}_page1.png")
        page.save(image_path, 'PNG')
        print(f"[PDF] Première page convertie : {image_path}")
        img = cv2.imread(image_path)
    else:
        img = cv2.imread(file_path)

    if img is None:
        print("Erreur : Impossible de lire l'image.")
        return

    # Prétraitement
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)
    denoised = cv2.GaussianBlur(binary, (5, 5), 0)

    # Sauvegarde
    output_path = os.path.join(output_folder, f"{name}_preprocessed_cleaned.png")
    cv2.imwrite(output_path, denoised)
    print(f"Image prétraitée et sauvegardée : {output_path}")

    # Affichage
    Image.fromarray(denoised).show()

#Analyse et nettoyage du texte extrait
def extraire_texte_depuis_image(filepath):
    image = cv2.imread(filepath)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    binarized = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
    denoised_image = cv2.GaussianBlur(binarized, (5, 5), 0)
    preprocessed_image = Image.fromarray(denoised_image)
    preprocessed_image.save('facture_preprocessed_cleaned.png')

    pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
    image = Image.open('facture_preprocessed_cleaned.png')

    text = pytesseract.image_to_string(image, lang=TESSERACT_LANG)
    text = text.replace('Gesamtsumme', 'Gesamt').replace('Bruttowert', 'Gesamt')
    return text

def detect_champs_allemand(text):
    sections = {
        "verkäufer": [],
        "kunde": [],
        "iban": None,
        "ust_ids": [],
        "artikel": []
    }

    current_section = None
    artikel_start = False

    for line in text.split('\n'):
        line_clean = line.strip().lower()

        # Détection des sections
        if "verkäufer" in line_clean:
            current_section = "verkäufer"
            continue
        elif "kunde" in line_clean:
            current_section = "kunde"
            continue
        elif "artikel" in line_clean:
            artikel_start = True
            continue
        elif "zusammenfassung" in line_clean:
            artikel_start = False
            continue

        # Extraction IBAN/USt-ID
        if "iban" in line_clean:
            iban_match = re.search(r'IBAN[\s:]*([A-Z]{2}\d{2}[\sA-Z0-9]{10,30})', line, re.IGNORECASE)
            if iban_match:
                sections["iban"] = iban_match.group(1).replace(' ', '')

        ust_match = re.search(r'(USt IdNr|Umsatzsteuer-ID)[\s:]*([0-9]{3}-[0-9]{2}-[0-9]{4})', line)
        if ust_match:
            sections["ust_ids"].append(ust_match.group(2))

        # Collecte des données
        if current_section:
            sections[current_section].append(line.strip())
        elif artikel_start and re.match(r'^\d+\.', line):
            sections["artikel"].append(line)

    # Traitement des données
    seller_data = "\n".join(sections["verkäufer"])
    client_data = "\n".join(sections["kunde"])

    # Séparation nom/adresse
    seller_name = sections["verkäufer"][0] if sections["verkäufer"] else ""
    seller_address = "\n".join(sections["verkäufer"][1:]) if len(sections["verkäufer"]) > 1 else ""

    client_name = sections["kunde"][0] if sections["kunde"] else ""
    client_address = "\n".join(sections["kunde"][1:]) if len(sections["kunde"]) > 1 else ""

    return {
        'seller_name': seller_name,
        'seller_address': seller_address,
        'seller_ust': sections["ust_ids"][0] if len(sections["ust_ids"]) > 0 else "",
        'client_name': client_name,
        'client_address': client_address,
        'client_ust': sections["ust_ids"][1] if len(sections["ust_ids"]) > 1 else "",
        'iban': sections["iban"],
        'artikel': sections["artikel"]
    }

# Fonction d'analyse principale
def analyse_facture_allemande(filepath):
    # Extraction du texte depuis l'image/PDF
    text = extraire_texte_depuis_image(filepath)
    print("[DEBUG] Texte extrait:\n", text[:500] + "...")  # Aperçu du texte

    # =============================================================
    # 1. Extraction du numéro de facture
    # =============================================================
    rechnungsnr_match = re.search(r'Rechnungsnummer[\s:]*([A-Z0-9-]+)', text, re.IGNORECASE)
    rechnungsnr = rechnungsnr_match.group(1) if rechnungsnr_match else None

    # =============================================================
    # 2. Traitement de la date
    # =============================================================
    datum = None
    date_match = re.search(r'Ausstellungsdatum[\s:\n]*(\d{2}[./-]\d{2}[./-]\d{4})', text)

    if date_match:
        raw_date = date_match.group(1)
        date_formats = [
            "%d.%m.%Y", "%d/%m/%Y", "%d-%m-%Y",
            "%m/%d/%Y", "%Y-%m-%d", "%d.%m.%y"
        ]

        for fmt in date_formats:
            try:
                parsed_date = datetime.strptime(raw_date, fmt)
                datum = parsed_date.strftime("%Y-%m-%d")
                break
            except ValueError:
                continue

    # =============================================================
    # 3. Extraction du montant total
    # =============================================================
    total = 0.0
    total_match = re.search(
        r'(Gesamt|Endbetrag|Bruttowert|Summe)[\s€:*]*(?:[\s€]*)([\d\s.,]+)',
        text.replace('\n', ' '),
        re.IGNORECASE
    )

    if total_match:
        amount_str = total_match.group(2)
        try:
            # Conversion format allemand
            total = float(
                amount_str.strip()
                .replace(' ', '')
                .replace('.', '')
                .replace(',', '.')
            )
            print(f"[SUCCÈS] Montant converti : {total}")
        except Exception as e:
            print(f"[ERREUR] Conversion montant '{amount_str}': {str(e)}")

    # =============================================================
    # 4. Extraction IBAN (version améliorée)
    # =============================================================
    iban = None
    iban_match = re.search(
        r'IBAN[\s:]*((?:[A-Z]{2}\d{2}[\sA-Z0-9]{10,30}))',
        text,
        re.IGNORECASE
    )

    if iban_match:
        iban = iban_match.group(1)\
              .replace(' ', '')\
              .replace('\n', '')\
              .strip()[:34]
        print(f"[SUCCÈS] IBAN détecté : {iban}")

    # =============================================================
    # 5. Extraction vendeur/client
    # =============================================================
    def parse_entity(lines):
        """Extrait nom, adresse et USt-ID d'un bloc de texte"""
        if not lines:
            return {'name': '', 'address': '', 'ust_id': ''}

        # Trouver USt-ID avec regex améliorée
        ust_match = re.search(r'\b\d{3}-\d{2}-\d{4}\b', lines[-1])
        if ust_match:
            ust_id = ust_match.group()
            address_lines = lines[:-1]
        else:
            ust_id = ''
            address_lines = lines

        return {
            'name': address_lines[0] if len(address_lines) > 0 else '',
            'address': '\n'.join(address_lines[1:]) if len(address_lines) > 1 else '',
            'ust_id': ust_id
        }

    # Extraction vendeur
    seller_section = re.search(r'Verkäufer:?(.*?)(?=\n[A-Z]{2,})', text, re.DOTALL | re.IGNORECASE)
    seller_lines = [line.strip() for line in seller_section.group(1).split('\n') if line.strip()] if seller_section else []
    seller = parse_entity(seller_lines)

    # Extraction client
    client_section = re.search(r'Kunde:?(.*?)(?=\n[A-Z]{2,})', text, re.DOTALL | re.IGNORECASE)
    client_lines = [line.strip() for line in client_section.group(1).split('\n') if line.strip()] if client_section else []
    client = parse_entity(client_lines)


    # =============================================================
    # 6. Gestion base de données
    # =============================================================
    try:
        conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password="ayaDATA2025@",
            database="extraction_factures"
        )
        cursor = conn.cursor()

        # Insertion vendeur avec données extraites
        cursor.execute("""
            INSERT INTO verkaeufer_de (name, adresse, ust_id)
            VALUES (%s, %s, %s)
            ON DUPLICATE KEY UPDATE id=id
        """, (seller['name'], seller['address'], seller['ust_id']))

        # Insertion client avec données extraites
        cursor.execute("""
            INSERT INTO kunden_de (name, adresse, ust_id)
            VALUES (%s, %s, %s)
            ON DUPLICATE KEY UPDATE id=id
        """, (client['name'], client['address'], client['ust_id']))

        # Récupération des IDs
        cursor.execute("SELECT id FROM verkaeufer_de WHERE ust_id = %s", (seller['ust_id'],))
        verkaeufer_id = cursor.fetchone()[0]

        cursor.execute("SELECT id FROM kunden_de WHERE ust_id = %s", (client['ust_id'],))
        kunde_id = cursor.fetchone()[0]

        # Insertion facture avec toutes les données
        cursor.execute("""
            INSERT INTO rechnungen_de
            (rechnungsnummer, datum, iban, gesamt, verkaeufer_id, kunde_id)
            VALUES (%s, %s, %s, %s, %s, %s)
        """, (
            rechnungsnr,
            datum if datum else None,
            iban if iban else None,
            total,
            verkaeufer_id,
            kunde_id
        ))

        conn.commit()
        print("[SUCCÈS] Données enregistrées en base")

    except mysql.connector.Error as err:
        print(f"[ERREUR] MySQL: {err}")
        if conn.is_connected():
            conn.rollback()
    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()

    return {
        'rechnungsnummer': rechnungsnr,
        'datum': datum,
        'iban': iban,
        'gesamt': total,
        'verkaeufer': seller['name'],
        'kunde': client['name']
    }
# Interface utilisateur adaptée
class ApplicationDE(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Rechnungsanalyse (DE)")
        self.geometry("600x400")

        style = ttk.Style()
        style.theme_use("clam")

        self.create_widgets()

    def create_widgets(self):
        ttk.Label(self, text="Deutsche Rechnungsanalyse", font=("Helvetica", 16)).pack(pady=10)

        self.btn_choose = ttk.Button(self, text="Rechnung auswählen", command=self.analysieren)
        self.btn_choose.pack(pady=5)

        self.status = ttk.Label(self, text="Bereit")
        self.status.pack(pady=5)

    def analysieren(self):
        filepath = choose_file()
        if filepath:
            try:
                analyse_facture_allemande(filepath)
                self.status.config(text="Erfolgreich analysiert!")
            except Exception as e:
                self.status.config(text=f"Fehler: {str(e)}")

if __name__ == "__main__":
    app = ApplicationDE()
    app.mainloop()

[DEBUG] Texte extrait:
 Rechnungsnummer: 72005605

Ausstellungsdatum:

09/16/2019
——j;_ Verkaufer: Kunde:
Butler. and Sons Pearson Group
355 Caitlin Centers Apt. 762 6544 Charles Isle
Amyview, TX 81463 © Jeffreyhaven, WV 26330

USt IdNr: 903-99-0661

USt IdNr: 909-98-4977
IBAN: GB80)JXBS9861 1986795347

——'_ ARTIKEL.

Nr. Beschreibung ME Nettopreis Netto-Wert Umsatzsteuer aa |

[%] Wert

6'x3' Eleganter Marmor-Tablett Tisch 7 Of 5 46% 38 '10 938,76

Intarsien-Mébeldekoration £9648

12032,64'

2 7x4" Sammierwertiger Mar...
[ERREUR] MySQL: Unread result found
[DEBUG] Texte extrait:
 ' Rechnungsnummer: 14778207

* Ausstellungsdatum: 01/06/2017:

— Verkaufer: Kunde:
Allen inc Caldwell, Ortega and Adams
205 Glenn Fork
3344 Candace Crossroad Ai ‘ sis DOSS
Harmonton, FL 43347 yssaberg,
USt IdNr; 989-70-9478 USt IdNr; 924-91-4797

IBAN: GB30DMSA405540667 13364

—' ARTIKEL

Nr. Beschreibung ME Nettopreis Netto-Wert Umsatzsteuer Brutto
[%] Wert

24°x24" WeiBer, Marmor-Tisch mit 18"

hoherMukifar

KeyboardInterrupt: 

In [ ]:
import pytesseract

# Chemin vers l'exécutable de Tesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

